In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from datetime import timedelta
import requests

In [4]:
import requests
from bs4 import BeautifulSoup
all_event_links_df = pd.DataFrame()
for year in range(2015, 2024):
    url = f"https://www.autosport.com/f1/results/{year}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    event_links = []
    class_name = "ms-select__option ms-select__option--event"
    for page in soup.find_all('a', attrs = {'class':class_name}):
        link = page.get('href')
        # print(link)
        elink = url.replace(f"/f1/results/{year}", link)
        # print(url)
        event_links.append(elink)
    print(len(event_links))
    rounds = [x for x in range(1, len(event_links)+1)]
    event_links_df = pd.DataFrame({
        "season": year,
        "round": rounds,
        "event_link": event_links
    })
    # display(event_links_df)
    all_event_links_df = pd.concat([all_event_links_df, event_links_df], axis=0).reset_index(drop=True)
    all_event_links_df = all_event_links_df.iloc[:len(all_event_links_df)-1]
display(all_event_links_df)
# all_event_links_df.to_csv("./data/autosport_links_error.csv", index=False)

18
20
19
20
20
16
22
23
22


,season,round,event_link
0,2015,1,https://www.autosport.com/f1/results/2015/aust...
1,2015,2,https://www.autosport.com/f1/results/2015/mala...
2,2015,3,https://www.autosport.com/f1/results/2015/chin...
3,2015,4,https://www.autosport.com/f1/results/2015/bahr...
4,2015,5,https://www.autosport.com/f1/results/2015/span...
...,...,...,...
166,2023,17,https://www.autosport.com/f1/results/2023/qata...
167,2023,18,https://www.autosport.com/f1/results/2023/unit...
168,2023,19,https://www.autosport.com/f1/results/2023/mexi...
169,2023,20,https://www.autosport.com/f1/results/2023/braz...


In [52]:
final_df = pd.read_csv('./data/final_df.csv')

print(f"qualifying_position: {final_df['qualifying_position'].isnull().sum()}")
print(f"qualifying_time: {final_df['qualifying_time'].isnull().sum()}")
print(f"sprint_qualifying_position: {final_df['sprint_qualifying_position'].isnull().sum()}")
print(f"sprint_qualifying_time: {final_df['sprint_qualifying_time'].isnull().sum()}")
print(f"fp_pos_1: {final_df['fp_pos_1'].isnull().sum()}")
print(f"fp_time_1: {final_df['fp_time_1'].isnull().sum()}")
print(f"fp_pos_2: {final_df['fp_pos_2'].isnull().sum()}")
print(f"fp_time_2: {final_df['fp_time_2'].isnull().sum()}")
print(f"fp_pos_3: {final_df['fp_pos_3'].isnull().sum()}")
print(f"fp_time_3: {final_df['fp_time_3'].isnull().sum()}")
print(f"sprint_fl_pos: {final_df['sprint_fl_pos'].isnull().sum()}")
print(f"sprint_fl_time: {final_df['sprint_fl_time'].isnull().sum()}")
print(f"sprint_laps: {final_df['sprint_laps'].isnull().sum()}")
print(f"sprint_position: {final_df['sprint_position'].isnull().sum()}")
print(f"sprint_time: {final_df['sprint_time'].isnull().sum()}")
print(f"fl_pos: {final_df['fl_pos'].isnull().sum()}")
print(f"fl_time: {final_df['fl_time'].isnull().sum()}")
# final_df.head(1)

qualifying_position: 171
qualifying_time: 316
sprint_qualifying_position: 3581
sprint_qualifying_time: 3591
fp_pos_1: 174
fp_time_1: 296
fp_pos_2: 257
fp_time_2: 324
fp_pos_3: 323
fp_time_3: 411
sprint_fl_pos: 3603
sprint_fl_time: 3603
sprint_laps: 3580
sprint_position: 3580
sprint_time: 3582
fl_pos: 283
fl_time: 301


In [ ]:
def time_cleanup(input_string):
    result_string = np.nan
    try:
        groups = input_string.split(" ")
        if len(groups) == 1:
            result_string = groups[0]
        if len(groups) == 2:
            result_string = groups[1]
        if len(groups) == 3:
            result_string = groups[2]
    except:
        print("time cleanup error")
    return result_string

def convert_time(time_str):
    rounded = np.nan
    try:
        # print(time_str)
        # Splitting the time into minutes and seconds

        # minutes, seconds = time_str.split("'")
        time_strings = time_str.split("'")
        if len(time_strings) == 1:
            hours = 0
            minutes = 0
            seconds = time_strings[0]
        elif len(time_strings) == 2:
            hours = 0
            minutes = time_strings[0]
            seconds = time_strings[1]
        elif len(time_strings) == 3:
            hours = time_strings[0]
            minutes = time_strings[1]
            seconds = time_strings[2]

        total_seconds = int(hours) * 60 * 60 + int(minutes) * 60 + float(seconds)
        # Converting total seconds to hh:mm:ss format
        formatted_time = timedelta(seconds=total_seconds)
        # Formatting the output to round milliseconds to 3 decimal places
        rounded = '{:02}:{:02}:{:06.3f}'.format(int(formatted_time.total_seconds() // 3600),
                                            int(formatted_time.total_seconds() % 3600 // 60),
                                            formatted_time.total_seconds() % 60)
        # print(rounded)
    except:
        print("time convert error")
    # print(rounded)
    return rounded

def time_to_milliseconds(time_str):
    milliseconds = np.nan
    try:
        parts = time_str.split(':')
        milliseconds = 0

        if len(parts) == 3:
            # Format is hours:minutes:seconds.milliseconds
            hours, minutes, seconds = parts
            milliseconds += int(hours) * 3600000  # Convert hours to milliseconds
            milliseconds += int(minutes) * 60000  # Convert minutes to milliseconds
        elif len(parts) == 2:
            # Format is minutes:seconds.milliseconds
            minutes, seconds = parts
            milliseconds += int(minutes) * 60000  # Convert minutes to milliseconds
        else:
            raise ValueError("Invalid time format")

        # Splitting seconds and milliseconds and converting
        seconds, msec = seconds.split('.')
        milliseconds += int(seconds) * 1000  # Convert seconds to milliseconds
        milliseconds += int(msec)  # Add milliseconds
    except:
        print("time to milli error")
    return milliseconds

def remove_space(input):
    result = str(input)
    result = result.replace(" ", "")
    return result

# def to_int(input):
#     return int(input)
def to_int(input):
    result = np.nan
    try:
        result = int(input)
    except:
        print("error when casting to int")
    return result

def get_tyre_type(input):
    return input.split(":")[0]

In [ ]:
url = "https://www.autosport.com/f1/results/2016/british-gp-488120/"

season = 2016
round = 10
race = pd.DataFrame()

try:
    qp_url = f"{url}?st=GRID"
    print(qp_url)
    # Get driver names
    r = requests.get(qp_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    driver_names = []
    class_name = "ms-link info-wrapper"
    for page in soup.find_all('a', attrs = {'class':class_name}):
        link = page.get('href')
        name = link.split("/")[2]
        name = name.replace("-", " ")
        name = name.lower()
        driver_names.append(name)
    name_df = pd.DataFrame({"driver_name": driver_names})
    table_df = pd.DataFrame()
    try: 
        table_df = pd.read_html(qp_url)
        table_df = table_df[0]
        table_df.columns = table_df.columns.droplevel(0)
        table_df = table_df[['Cla', 'Time']]
        table_df['Cla'] = table_df['Cla'].apply(to_int)
        table_df['Time'] = table_df['Time'].apply(time_cleanup)
        table_df['Time'] = table_df['Time'].apply(convert_time)
        table_df['Time'] = table_df['Time'].apply(time_to_milliseconds)
    except:
        print("no data found under tab GRID, try GRID1 for time and GRID2 for position")
        try:
            qp_time_url = f"{url}?st=GRID1"
            table_time_df = pd.read_html(qp_time_url)
            table_time_df = table_time_df[0]
            table_time_df.columns = table_time_df.columns.droplevel(0)
            table_time_df = table_time_df[['Time']]
            table_time_df['Time'] = table_time_df['Time'].apply(time_cleanup)
            table_time_df['Time'] = table_time_df['Time'].apply(convert_time)
            table_time_df['Time'] = table_time_df['Time'].apply(time_to_milliseconds)

        except:
            print("no Time data found under tab GRID1")

        try:
            qp_position_url = f"{url}?st=GRID2"
            table_pos_df = pd.read_html(qp_position_url)
            table_pos_df = table_pos_df[0]
            table_pos_df.columns = table_pos_df.columns.droplevel(0)
            table_pos_df = table_pos_df[['Cla']]
            table_pos_df['Cla'] = table_pos_df['Cla'].apply(to_int)
        except:
            print("no Position data found under tab GRID2")

        table_df = pd.concat([table_pos_df, table_time_df], axis=1)
        # continue
    
    table_df.rename(columns = {'Cla': "qualifying_position", 'Time':"qualifying_time"}, inplace = True)
    df = pd.concat([name_df, table_df], axis=1)
    df['season'] = season
    df['round'] = round
    display(table_df)
    # df['qp_url'] = qp_url
    # display(df)
    if race.empty:
        race = df
    else:
        race = pd.merge(race, df, how='outer', on=['season', 'round', 'driver_name'])
except:
    print("no data")

In [ ]:
# # This scraper has deprecated or we may have been denied access. Do not run this snippet of code or it will purge our existing qualifying_results.csv 
# constructor_standings = pd.read_csv("./data/constructor_standings.csv")
# # ==========
# # Qualifying Results
# # ==========
# from bs4 import BeautifulSoup
# def time_to_milliseconds(time_str):
#     milliseconds = np.nan
#     try:
#         parts = time_str.split(':')
#         milliseconds = 0
#         if len(parts) == 3:
#             # Format is hours:minutes:seconds.milliseconds
#             hours, minutes, seconds = parts
#             milliseconds += int(hours) * 3600000  # Convert hours to milliseconds
#             milliseconds += int(minutes) * 60000  # Convert minutes to milliseconds
#         elif len(parts) == 2:
#             # Format is minutes:seconds.milliseconds
#             minutes, seconds = parts
#             milliseconds += int(minutes) * 60000  # Convert minutes to milliseconds
#         else:
#             raise ValueError("Invalid time format")
#         # Splitting seconds and milliseconds and converting
#         seconds, msec = seconds.split('.')
#         milliseconds += int(seconds) * 1000  # Convert seconds to milliseconds
#         milliseconds += int(msec)  # Add milliseconds
#     except:
#         print("time to milli error")
#     return milliseconds

# def reformat_name(name):
#     name = name.replace("-", " ")
#     name = name.lower()
#     return name

# qualifying_results = pd.DataFrame()
# # Qualifying times are only available from 1983
# for year in list(range(2021,2022)):
#     url = 'https://www.formula1.com/en/results.html/{}/races.html'
#     r = requests.get(url.format(year))
#     soup = BeautifulSoup(r.text, 'html.parser')
#     # find links to all circuits for a certain year
#     year_links = []
#     for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
#         link = page.get('href')
#         if f'/en/results.html/{year}/races/' in link: 
#             year_links.append(link)
#     # for each circuit, switch to the starting grid page and read table
#     year_df = pd.DataFrame()
#     new_url = 'https://www.formula1.com{}'
#     for n, link in list(enumerate(year_links)):
#         link = link.replace('race-result.html', 'starting-grid.html')
#         print(f"{year}, {n+1}")
#         print(f"https://www.formula1.com{link}")
#         try:
#             df = pd.read_html(new_url.format(link))
#         except:
#             print("data not yet available")
#             continue
#         df = df[0]
#         display(df)
#         df['season'] = year
#         df['round'] = n+1
#         for col in df:
#             if 'Unnamed' in col:
#                 df.drop(col, axis = 1, inplace = True)
#         year_df = pd.concat([year_df, df])
#     # concatenate all tables from all years  
#     qualifying_results = pd.concat([qualifying_results, year_df])
# # rename columns
# qualifying_results.rename(columns = {'Pos': 'grid', 'Driver': 'driver_name', 'Car': 'car',
#                                      'Time': 'qualifying_time'}, inplace = True)
# # drop driver number column
# # qualifying_results.drop(['No', 'Gap', 'Laps'], axis = 1, inplace = True)
# qualifying_results.drop(['No'], axis = 1, inplace = True)
# qualifying_results['driver_name'] = qualifying_results['driver_name'].str.split().apply(lambda x: ' '.join(x[:-1]))
# qualifying_results['driver_name'] = qualifying_results['driver_name'].apply(reformat_name)
# qualifying_results['qualifying_time'] = qualifying_results['qualifying_time'].apply(time_to_milliseconds)
# qualifying_results.to_csv("./data/qualifying_results.csv", sep=",", index=False)
# display(qualifying_results)
# for row in qualifying_results.itertuples():
#     print(f"{row.season}, {row.round}, {row.qualifying_time}")